[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_sdk/pdftools_sdk_encrypt.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDK/latest/pdftools_sdk-latest.tar.gz
%pip install ipython

# Encrypt a PDF
Encrypt a PDF with a user password.
To open the document, the user password is required.

In [ ]:
import io
from pdftools_sdk.pdf import Document, Encryption, Permission
from pdftools_sdk.sign import OutputOptions as SignOutputOptions, SignatureRemoval, Signer

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
password = 'asdf'
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/InvoiceNone.pdf'
input_path = 'InvoiceNone.pdf'
prepare_file(input_url, input_path)
output_path = 'Encrypted.pdf'

In [ ]:
def encrypt(password: str, input_path: str, output_path: str):
    # Open input document
    with io.FileIO(input_path, 'rb') as in_stream:
        with Document.open(in_stream) as input_document:
            # Create stream for output file
            with io.FileIO(output_path, 'wb+') as output_stream:
                # Set encryption options
                output_options = SignOutputOptions()
                # Set a user password that will be required to open the document.
                # Note that this will remove PDF/A conformance of input files (see warning category Sign.WarningCategory.PdfARemoved)
                output_options.encryption = Encryption(password, None, Permission.ALL)
                # Allow removal of signatures. Otherwise the Encryption property is ignored for signed input documents
                # (see warning category Sign.WarningCategory.SignedDocEncryptionUnchanged).
                output_options.remove_signatures = SignatureRemoval.SIGNED

                # Encrypt the document
                signer = Signer()
                signer.process(input_document, output_stream, output_options)

In [ ]:
try:
    # By default, a test license key is active. In this case, a watermark is added to the output. 
    # If you have a license key, please uncomment the following call and set the license key.
    # from pdftools_sdk.sdk import Sdk
    # Sdk.initialize("INSERT-LICENSE-KEY")

    # Encrypt a PDF document
    encrypt(password, input_path, output_path)

    print(f"Successfully created encrypted {output_path}")
except Exception as e:
    print(f"An error occurred: {e}")